<a href="https://colab.research.google.com/github/Mrcocvs02/Dissertation/blob/master/clifford_basis_states.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
num_qubits = 5
num_shots = 10000

num_samples = 10

In [ ]:
pip install qiskit qiskit_ibm_runtime qiskit_aer

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer import AerSimulator

service = QiskitRuntimeService(
    channel="ibm_quantum",
    token="b5db7ca06768cddc25c6a333037ab8c57800a6ba19b33fc6e673ecc0320f890345fe94daf93ffa6b71fec434835d6deb393ce34cee00ebadbc3fc9f6c6789248",
)

# Qiskit Clifford simulator
ideal_simulator = AerSimulator(method="stabilizer")

# Qiskit noise model simulator
real_device = service.backend("ibm_sherbrooke")
noisy_simulator = AerSimulator.from_backend(real_device)

In [ ]:
import random
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit import QuantumCircuit

pm = generate_preset_pass_manager(backend=ideal_simulator, optimization_level=1)

random.seed(1)
circuits = []

for _ in range(num_samples):
    qc = QuantumCircuit(num_qubits)
    for qubit in range(num_qubits):
        if random.random() < 0.5:
            qc.x(qubit)
    qc.measure_all()
    circuits.append(pm.run(qc))

In [ ]:
from qiskit_ibm_runtime import SamplerV2 as Sampler

ideal_sampler = Sampler(backend=ideal_simulator)
ideal_job = ideal_sampler.run(circuits, shots=num_shots)
ideal_results = [res.data.meas.get_counts() for res in ideal_job.result()]

In [ ]:
noisy_sampler = Sampler(backend=noisy_simulator)
noisy_job = noisy_sampler.run(circuits, shots=num_shots)
noisy_results = [res.data.meas.get_counts() for res in noisy_job.result()]

In [ ]:
from qiskit.visualization import plot_distribution

idx = 0

plot_distribution(
    [ideal_results[idx], noisy_results[idx]],
    legend=["Ideal", "Noisy"],
    bar_labels=False,
)

In [ ]:
circuits[idx].draw("mpl")